In [1]:
import open3d as o3d
from numba.core.errors import NumbaDeprecationWarning,NumbaPendingDeprecationWarning, NumbaWarning
import warnings
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaWarning)
import sys
sys.path.append('/kaggle/code/ConeDetectionPointpillars')

from second.data.CustomNuscDataset import * #to register dataset
from models import * #to register model
import torch
from second.utils.log_tool import SimpleModelLog
from second.builder import target_assigner_builder, voxel_builder
from second.protos import pipeline_pb2
from second.pytorch.builder import (box_coder_builder, input_reader_builder,
                                    lr_scheduler_builder, optimizer_builder,
                                    second_builder)
from second.pytorch.core import box_torch_ops
import torchplus
import re
import time
from pathlib import Path
import numpy as np
import pandas as pd
import json
from collections import defaultdict
from google.protobuf import text_format
# from lyft_dataset_sdk.utils.geometry_utils import *
from lyft_dataset_sdk.lyftdataset import Quaternion
# from lyft_dataset_sdk.utils.data_classes import Box
from nuscenes.utils.geometry_utils import *
from nuscenes.utils.data_classes import Box
from second.utils.progress_bar import ProgressBar
#commented second.core.non_max_suppression, nms_cpu, __init__.py, 
# pytorch.core.box_torch_ops line 524

tensor([[-1.1507, -0.9086]])
tensor([[-2.0003, -1.2439]], device='cuda:0')
tensor([[-1.1507, -0.9086]])
False
tensor([[-1.1507, -0.9086]], device='cuda:0')
True
True


In [2]:
def build_network(model_cfg, measure_time=False):
    voxel_generator = voxel_builder.build(model_cfg.voxel_generator)
    bv_range = voxel_generator.point_cloud_range[[0, 1, 3, 4]]
    box_coder = box_coder_builder.build(model_cfg.box_coder)
    target_assigner_cfg = model_cfg.target_assigner
    target_assigner = target_assigner_builder.build(target_assigner_cfg,
                                                    bv_range, box_coder)
    box_coder.custom_ndim = target_assigner._anchor_generators[0].custom_ndim
    net = second_builder.build(
        model_cfg, voxel_generator, target_assigner, measure_time=measure_time)
    return net


def merge_second_batch_multigpu(batch_list):
    if isinstance(batch_list[0], list):
        batch_list_c = []
        for example in batch_list:
            batch_list_c += example
        batch_list = batch_list_c

    example_merged = defaultdict(list)
    for example in batch_list:
        for k, v in example.items():
            example_merged[k].append(v)
    ret = {}
    for key, elems in example_merged.items():
        if key == 'metadata':
            ret[key] = elems
        elif key == "calib":
            ret[key] = {}
            for elem in elems:
                for k1, v1 in elem.items():
                    if k1 not in ret[key]:
                        ret[key][k1] = [v1]
                    else:
                        ret[key][k1].append(v1)
            for k1, v1 in ret[key].items():
                ret[key][k1] = np.stack(v1, axis=0)
        elif key == 'coordinates':
            coors = []
            for i, coor in enumerate(elems):
                coor_pad = np.pad(
                    coor, ((0, 0), (1, 0)), mode='constant', constant_values=i)
                coors.append(coor_pad)
            ret[key] = np.stack(coors, axis=0)
        elif key in ['gt_names', 'gt_classes', 'gt_boxes']:
            continue
        else:
            ret[key] = np.stack(elems, axis=0)

    return ret

def buildBBox(points,color = [1,0,0]):
    #print("Let's draw a cubic using o3d.geometry.LineSet")
    # points = [[0, 0, 0], [1, 0, 0], [0, 1, 0], [1, 1, 0], [0, 0, 1], [1, 0, 1],
    #           [0, 1, 1], [1, 1, 1]] x0y0z0, x0y0z1, x0y1z0, x0y1z1, x1y0z0, x1y0z1, x1y1z0, x1y1z1

    points = points[[0,4,3,7,1,5,2,6],:]
    lines = [[0, 1], [0, 2], [1, 3], [2, 3], [4, 5], [4, 6], [5, 7], [6, 7],
             [0, 4], [1, 5], [2, 6], [3, 7]]
    colors = [color for i in range(len(lines))]
    line_set = o3d.geometry.LineSet()
    line_set.points = o3d.utility.Vector3dVector(points)
    line_set.lines = o3d.utility.Vector2iVector(lines)
    line_set.colors = o3d.utility.Vector3dVector(colors)


    return  line_set
def merge_second_batch(batch_list):
    if isinstance(batch_list[0], list):
        batch_list_c = []
        for example in batch_list:
            batch_list_c += example
        batch_list = batch_list_c

    example_merged = defaultdict(list)
    for example in batch_list:
        for k, v in example.items():
            example_merged[k].append(v)
    ret = {}
    for key, elems in example_merged.items():
        if key in [
            'voxels', 'num_points', 'num_gt', 'voxel_labels', 'gt_names', 'gt_classes', 'gt_boxes'
        ]:
            ret[key] = np.concatenate(elems, axis=0)
        elif key == 'metadata':
            ret[key] = elems
        elif key == "calib":
            ret[key] = {}
            for elem in elems:
                for k1, v1 in elem.items():
                    if k1 not in ret[key]:
                        ret[key][k1] = [v1]
                    else:
                        ret[key][k1].append(v1)
            for k1, v1 in ret[key].items():
                ret[key][k1] = np.stack(v1, axis=0)
        elif key == 'coordinates':
            coors = []
            for i, coor in enumerate(elems):
                coor_pad = np.pad(
                    coor, ((0, 0), (1, 0)), mode='constant', constant_values=i)
                coors.append(coor_pad)
            ret[key] = np.concatenate(coors, axis=0)
        elif key == 'metrics':
            ret[key] = elems
        else:
            ret[key] = np.stack(elems, axis=0)
    return ret


def _worker_init_fn(worker_id):
    time_seed = np.array(time.time(), dtype=np.int32)
    np.random.seed(time_seed + worker_id)


def example_convert_to_torch(example, dtype=torch.float32,
                             device=None) -> dict:
    device = device or torch.device("cuda:0")
    example_torch = {}
    float_names = [
        "voxels", "anchors", "reg_targets", "reg_weights", "bev_map", "importance"
    ]
    for k, v in example.items():
        if k in ['gt_names', 'gt_classes', 'gt_boxes', 'points']:
            example_torch[k] = example[k]
            continue

        if k in float_names:
            # slow when directly provide fp32 data with dtype=torch.half
            example_torch[k] = torch.tensor(
                v, dtype=torch.float32, device=device).to(dtype)
        elif k in ["coordinates", "labels", "num_points"]:
            example_torch[k] = torch.tensor(
                v, dtype=torch.int32, device=device)
        elif k in ["anchors_mask"]:
            example_torch[k] = torch.tensor(
                v, dtype=torch.uint8, device=device)
        elif k == "calib":
            calib = {}
            for k1, v1 in v.items():
                calib[k1] = torch.tensor(
                    v1, dtype=dtype, device=device).to(dtype)
            example_torch[k] = calib
        elif k == "num_voxels":
            example_torch[k] = torch.tensor(v)
        else:
            example_torch[k] = v
    return example_torch


def time_to_str(t, mode='min'):
    if mode == 'min':
        t = int(t) / 60
        hr = t // 60
        min = t % 60
        return '%2d hr %02d m' % (hr, min)

    elif mode == 'sec':
        t = int(t)
        min = t // 60
        sec = t % 60
        return '%2d min %02d sec' % (min, sec)


    else:
        raise NotImplementedError


In [3]:
config_path = '/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874/pipeline.config'
# config_path = '/kaggle/code/ConeDetectionPointpillars/customNuscenes/configs/cones_pp_initial_v2.config'
# config_path = '/kaggle/code/ConeDetectionPointpillars/customNuscenes/configs/cones_pp_initialak.config'
model_dir = f'/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874'
result_path=None
create_folder=False
display_step=50
pretrained_path="/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874/29-1-2021_9:53/voxelnet-5850.tckpt"
multi_gpu=False
measure_time=False
resume=True


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_dir = str(Path(model_dir).resolve())
if create_folder:
    if Path(model_dir).exists():
        model_dir = torchplus.train.create_folder(model_dir)
model_dir = Path(model_dir)
if resume:
    cur_time = time.localtime(time.time())
    cur_time = f'{cur_time.tm_mday}-{cur_time.tm_mon}-{cur_time.tm_year}_{cur_time.tm_hour}:{cur_time.tm_min}'
    model_dir = model_dir / cur_time
    model_dir.mkdir(parents=True, exist_ok=True)
if not resume and model_dir.exists():
    raise ValueError("model dir exists and you don't specify resume")

model_dir.mkdir(parents=True, exist_ok=True)
if result_path is None:
    result_path = model_dir / 'results'
config_file_bkp = "pipeline.config"

if isinstance(config_path, str):
    config = pipeline_pb2.TrainEvalPipelineConfig()
    with open(config_path, "r") as f:
        proto_str = f.read()
        text_format.Merge(proto_str, config)
else:
    config = config_path
    proto_str = text_format.MessageToString(config, indent=2)

with (model_dir / config_file_bkp).open('w') as f:
    f.write(proto_str)
# Read config file
input_cfg = config.train_input_reader
eval_input_cfg = config.eval_input_reader
model_cfg = config.model.second  # model's config
train_cfg = config.train_config  # training config


In [4]:
# Build neural network
net = build_network(model_cfg, measure_time).to(device)

target_assigner = net.target_assigner
voxel_generator = net.voxel_generator
print("num parameter: ", len(list(net.parameters())))
torchplus.train.try_restore_latest_checkpoints(model_dir, [net])

num parameter:  64


In [5]:
if pretrained_path is not None:
    print('warning pretrain is loaded after restore, careful with resume')
    model_dict = net.state_dict()
    pretrained_dict = torch.load(pretrained_path)

    new_pretrained_dict = {}
    for k, v in pretrained_dict.items():
        if k in model_dict and v.shape == model_dict[k].shape:
            new_pretrained_dict[k] = v
    print("Load pretrained parameters: ")
    for k, v in new_pretrained_dict.items():
        print(k, v.shape)
    model_dict.update(new_pretrained_dict)
    net.load_state_dict(model_dict)
    net.clear_global_step()
    net.clear_metrics()
if multi_gpu:
    net_parallel = torch.nn.DataParallel(net)
else:
    net_parallel = net
optimizer_cfg = train_cfg.optimizer
loss_scale = train_cfg.loss_scale_factor
fastai_optimizer = optimizer_builder.build(
    optimizer_cfg,
    net,
    mixed=False,
    loss_scale=loss_scale)
if loss_scale < 0:
    loss_scale = "dynamic"
if train_cfg.enable_mixed_precision:
    max_num_voxels = input_cfg.preprocess.max_number_of_voxels * input_cfg.batch_size
    print("max_num_voxels: %d" % (max_num_voxels))
    from apex import amp
    net, amp_optimizer = amp.initialize(net, fastai_optimizer,
                                        opt_level="O1",
                                        keep_batchnorm_fp32=None,
                                        loss_scale=loss_scale)
    net.metrics_to_float()
else:
    amp_optimizer = fastai_optimizer
torchplus.train.try_restore_latest_checkpoints(model_dir, [fastai_optimizer])
lr_scheduler = lr_scheduler_builder.build(optimizer_cfg, amp_optimizer, train_cfg.steps)

if train_cfg.enable_mixed_precision:
    float_dtype = torch.float16
else:
    float_dtype = torch.float32

if multi_gpu:
    num_gpu = torch.cuda.device_count()
    print(f"MULTI_GPU: use {num_gpu} gpus")
    collate_fn = merge_second_batch_multigpu
else:
    collate_fn = merge_second_batch
    num_gpu = 1

warning pretrain is loaded after restore, careful with resume
Load pretrained parameters: 
global_step torch.Size([1])
voxel_feature_extractor.pfn_layers.0.linear.weight torch.Size([64, 10])
voxel_feature_extractor.pfn_layers.0.norm.weight torch.Size([64])
voxel_feature_extractor.pfn_layers.0.norm.bias torch.Size([64])
voxel_feature_extractor.pfn_layers.0.norm.running_mean torch.Size([64])
voxel_feature_extractor.pfn_layers.0.norm.running_var torch.Size([64])
voxel_feature_extractor.pfn_layers.0.norm.num_batches_tracked torch.Size([])
rpn.blocks.0.1.weight torch.Size([64, 64, 3, 3])
rpn.blocks.0.2.weight torch.Size([64])
rpn.blocks.0.2.bias torch.Size([64])
rpn.blocks.0.2.running_mean torch.Size([64])
rpn.blocks.0.2.running_var torch.Size([64])
rpn.blocks.0.2.num_batches_tracked torch.Size([])
rpn.blocks.0.4.weight torch.Size([64, 64, 3, 3])
rpn.blocks.0.5.weight torch.Size([64])
rpn.blocks.0.5.bias torch.Size([64])
rpn.blocks.0.5.running_mean torch.Size([64])
rpn.blocks.0.5.running_va

In [6]:
dataset = input_reader_builder.build(
    input_cfg,
    model_cfg,
    training=True,
    voxel_generator=voxel_generator,
    target_assigner=target_assigner,
    multi_gpu=multi_gpu
)
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=input_cfg.batch_size * num_gpu,
    shuffle=True,
    num_workers=input_cfg.preprocess.num_workers * num_gpu,
    pin_memory=False,
    collate_fn=collate_fn,
    worker_init_fn=_worker_init_fn,
    drop_last=not multi_gpu
)


feature_map_size [1, 400, 600]
Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 21.197 seconds.
Reverse indexing ...
Done reverse indexing in 5.3 seconds.


In [7]:
dataset[1]

INFO - 2021-01-29 17:31:17,128 - transforms - finding looplift candidates


{'voxels': array([[[ 1.07951928e+01, -5.69504188e-01, -3.43896583e-02,
          -4.33333333e-01,  0.00000000e+00],
         [ 1.07864057e+01, -5.08966396e-01, -3.20995047e-02,
          -4.41176471e-01,  0.00000000e+00],
         [ 1.07134437e+01, -5.79637576e-01, -3.38105015e-02,
          -4.41176471e-01,  4.97601032e-02],
         ...,
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00]],
 
        [[ 2.47440946e+01, -1.69396491e+01, -7.32347492e-01,
          -4.76470588e-01,  4.97601032e-02],
         [ 2.47943269e+01, -1.69241512e+01, -7.27681054e-01,
          -4.64705882e-01,  1.49842024e-01],
         [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
           0.00000000e+00,  0.00000000e+00],
         ...,
     

In [8]:
len(dataset)

3125

In [9]:
# # [line for line in dataset[0]['reg_targets'] for i in line if i != 0]
# res = []
# for line in dataset[1]['reg_targets']:
#     for i in line:
#         if i != 0:
#             print(line)
#             res.append(line)
#             break
# #     break
# res

In [10]:
len([i for i in dataset[1]['labels'] if i == 0])

203992

In [11]:
len(dataset[1]['labels'])

960000

In [12]:
len([i for i in dataset[1]['labels'] if i == -1])

757022

In [13]:
len([i for i in dataset[2]['labels'] if i == 1])

1705

In [14]:
len([i for i in dataset[10]['labels'] if i == 2])

1028

In [15]:
set([i for i in dataset[1]['labels'] if i != 0])

{-1, 1, 2}

In [16]:
# for i in range(len(dataset)):
#     if len(set([i for i in dataset[i]['labels'] if i != 0])) > 2:
#         print(i)
#         print(set([i for i in dataset[i]['labels'] if i != 0]))

In [17]:
eval_dataset = input_reader_builder.build(
    eval_input_cfg,
    model_cfg,
    training=False,
    voxel_generator=voxel_generator,
    target_assigner=target_assigner)
eval_dataloader = torch.utils.data.DataLoader(
    eval_dataset,
    batch_size=input_cfg.batch_size,
    shuffle=True,
    num_workers=eval_input_cfg.preprocess.num_workers,
    pin_memory=False,
    collate_fn=merge_second_batch)

feature_map_size [1, 400, 600]
Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 22.945 seconds.
Reverse indexing ...
Done reverse indexing in 5.1 seconds.


In [18]:
len(eval_dataset)

32

In [24]:
from tqdm import tqdm, tqdm_notebook
import time
model_logging = SimpleModelLog(model_dir)
model_logging.open()
model_logging.log_text(proto_str + "\n", 0, tag="config")

start_step = net.get_global_step()
total_step = train_cfg.steps
t = time.time()
steps_per_eval = train_cfg.steps_per_eval
clear_metrics_every_epoch = train_cfg.clear_metrics_every_epoch

amp_optimizer.zero_grad()
step_times = []
step = start_step
run = True
ave_valid_loss = 0.0

try:
    start_tic = time.time()
    print("num samples: %d" % (len(dataset)))
    while run == True:
        if clear_metrics_every_epoch:
            net.clear_metrics()
        for example in tqdm_notebook(dataloader):
            lr_scheduler.step(net.get_global_step())
            example.pop("metrics")
            example_torch = example_convert_to_torch(example, float_dtype)

            ret_dict = net_parallel(example_torch)
            loss = ret_dict["loss"].mean()
            cls_loss_reduced = ret_dict["cls_loss_reduced"].mean()
            loc_loss_reduced = ret_dict["loc_loss_reduced"].mean()

            if train_cfg.enable_mixed_precision:
                if net.get_global_step() < 100:
                    loss *= 1e-3
                with amp.scale_loss(loss, amp_optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 10.0)
            amp_optimizer.step()
            amp_optimizer.zero_grad()
            net.update_global_step()

            cls_preds = ret_dict["cls_preds"]
            labels = example_torch["labels"]
            cared = ret_dict["cared"]

            net_metrics = net.update_metrics(cls_loss_reduced,
                                             loc_loss_reduced, cls_preds,
                                             labels, cared)
            step_time = (time.time() - t)
            step_times.append(step_time)
            t = time.time()
            metrics = {}
            global_step = net.get_global_step()

            if global_step % display_step == 0:
                net.eval()
                det = net(example_torch)
                print(det[0]['label_preds'])
                print(det[0]['scores'])      
                net.train()
                eta = time.time() - start_tic
                if measure_time:
                    for name, val in net.get_avg_time_dict().items():
                        print(f"avg {name} time = {val * 1000:.3f} ms")

                metrics["step"] = global_step
                metrics['epoch'] = global_step / len(dataloader)
                metrics['steptime'] = np.mean(step_times)
                metrics['valid'] = ave_valid_loss
                step_times = []

                metrics["loss"] = net_metrics['loss']['cls_loss'] + net_metrics['loss']['loc_loss']
                metrics["cls_loss"] = net_metrics['loss']['cls_loss']
                metrics["loc_loss"] = net_metrics['loss']['loc_loss']

                if model_cfg.use_direction_classifier:
                    dir_loss_reduced = ret_dict["dir_loss_reduced"].mean()
                    metrics["dir_rt"] = float(
                        dir_loss_reduced.detach().cpu().numpy())

                metrics['lr'] = float(amp_optimizer.lr)
                metrics['eta'] = time_to_str(eta)
                model_logging.log_metrics(metrics, global_step)

                net.clear_metrics()
            if global_step % steps_per_eval == 0:
                torchplus.train.save_models(model_dir, [net, amp_optimizer],
                                            net.get_global_step())
                model_logging.log_text(f"Model saved: {model_dir}, {net.get_global_step()}", global_step)
                net.eval()
                result_path_step = result_path / f"step_{net.get_global_step()}"
                result_path_step.mkdir(parents=True, exist_ok=True)
                model_logging.log_text("########################", global_step)
                model_logging.log_text(" EVALUATE",global_step)
                model_logging.log_text("########################", global_step)
                model_logging.log_text("Generating eval images...", global_step)
                t = time.time()
                detections = []
                prog_bar = ProgressBar()
                net.clear_timer()
                cnt = 0
                for example in iter(dataloader):
                    cnt += 1
                    detection = example_convert_to_torch(example, float_dtype)
                    detection = net(detection)
#                     filtered_sample_tokens = eval_dataset.dataset.filtered_sample_tokens
                    filtered_sample_tokens = dataset.dataset.filtered_sample_tokens
                    index = filtered_sample_tokens.index(detection[0]['metadata']['token'])
                    det_boxes = detection[0]['box3d_lidar'].cpu().detach().numpy()
                    det_labels = detection[0]['label_preds'].cpu().detach().numpy()
                    det_scores = detection[0]['scores'].cpu().detach().numpy()
                    
                    det_boxes_cones = det_boxes[det_labels == 0]
                    det_scores_cones = det_scores[det_labels == 0]
                    det_labels_cones = det_labels[det_labels == 0]
                    
                    det_boxes_pedes = det_boxes[det_labels == 1]
                    det_scores_pedes = det_scores[det_labels == 1]
                    det_labels_pedes = det_labels[det_labels == 1]
#                     gt_example = eval_dataset.dataset.get_sensor_data(index)
                    gt_example = dataset.dataset.get_sensor_data(index)
                    points = gt_example['lidar']['points']
        
                    gt_boxes = gt_example['lidar']['annotations']['boxes']
                    gt_labels = gt_example['lidar']['annotations']['names']
                    gt_boxes_cones = gt_boxes[gt_labels == "traffic_cone"]
                    gt_labels_cones = gt_labels[gt_labels == "traffic_cone"]

                    gt_boxes_pedes = gt_boxes[gt_labels == "pedestrian"]
                    gt_labels_pedes = gt_labels[gt_labels == "pedestrian"]

                    gt_scores = np.ones(len(gt_labels_cones))
                    c = points[:, 3].reshape(-1, 1)
                    c = np.concatenate([c, c, c], axis=1)
                    points = points[:, 0:3]
                    pc = o3d.geometry.PointCloud()
                    pc.points = o3d.utility.Vector3dVector(points)
                    pc.colors = o3d.utility.Vector3dVector(c)
                    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2.0,
                                                                                   origin=[-0, -0, -0])
                    geo = [pc, mesh_frame]
                    rbbox_corners_cones = box_np_ops.center_to_corner_box3d(det_boxes_cones[:, :3],
                                                                      det_boxes_cones[:, 3:6],
                                                                      det_boxes_cones[:, 6],
                                                                      origin=(0.5, 0.5, 0.5), axis=2)
                    rbbox_corners_pedes = box_np_ops.center_to_corner_box3d(det_boxes_pedes[:, :3],
                                                                      det_boxes_pedes[:, 3:6],
                                                                      det_boxes_pedes[:, 6],
                                                                      origin=(0.5, 0.5, 0.5), axis=2)
                    gt_cones_rbbox_corners = box_np_ops.center_to_corner_box3d(gt_boxes_cones[:, :3],
                                                                         gt_boxes_cones[:, 3:6],
                                                                         gt_boxes_cones[:, 6],
                                                                         origin=(0.5, 0.5, 0.5), axis=2)
                    gt_pedes_rbbox_corners = box_np_ops.center_to_corner_box3d(gt_boxes_pedes[:, :3],
                                                                         gt_boxes_pedes[:, 3:6],
                                                                         gt_boxes_pedes[:, 6],
                                                                         origin=(0.5, 0.5, 0.5), axis=2)
                    for i in range(len(rbbox_corners_cones)):
                        geo.append(buildBBox(rbbox_corners_cones[i], color=[1, 0, 0]))
                    for i in range(len(rbbox_corners_pedes)):
                        geo.append(buildBBox(rbbox_corners_pedes[i], color=[1, 1, 0]))
                    for i in range(len(gt_cones_rbbox_corners)):
                        geo.append(buildBBox(gt_cones_rbbox_corners[i], color=[0, 1, 0]))
                    for i in range(len(gt_pedes_rbbox_corners)):
                        geo.append(buildBBox(gt_pedes_rbbox_corners[i], color=[0, 0, 1]))
#                     o3d.visualization.draw_geometries(geo)
                    vis = o3d.visualization.Visualizer()
                    vis.create_window(visible=True)
                    for i in range(len(geo)):
                        vis.add_geometry(geo[i])
                        vis.update_geometry(geo[i])
                    vis.poll_events()
                    vis.update_renderer()
                    img_dir = str(Path(f'{model_dir}/images/{global_step}').resolve())
                    img_dir = Path(img_dir)
                    img_dir.mkdir(parents=True, exist_ok=True)
                    vis.capture_screen_image(f'{model_dir}/images/{global_step}/eval_{cnt}.png')
                    model_logging.log_text(f"eval images saved at {model_dir}/images/{global_step}/", global_step)
                    vis.destroy_window()
                    if cnt >= 5:
                        break
                if 1 in detection[0]['label_preds']:
                    print(detection[0]['label_preds'])
                    print(detection[0]['scores'])
                sec_per_ex = len(dataset) / (time.time() - t)
                model_logging.log_text(
                    f'generate eval images finished({sec_per_ex:.2f}/s). continue training:',
                    global_step)
                net.train()
                
            step += 1
            if step >= total_step:
                break
        if step >= total_step:
            break
except Exception as e:
    model_logging.log_text(str(e), step)
    model_logging.log_text(json.dumps(example['metadata'], indent=2), step)
    torchplus.train.save_models(model_dir, [net, amp_optimizer], step)
    raise e
finally:
    model_logging.close()
    net.train()
torchplus.train.save_models(model_dir, [net, amp_optimizer], step)


model: {
  second: {
    network_class_name: "VoxelNet"
    voxel_generator {
      full_empty_part_with_mean: false
      point_cloud_range : [-30, -20, -2.6, 30, 20, 2.2] #[-80, -20, -2 , 60, 40, 3] # will use [0,1,3,4] means [-100,-100,100,1000]
      voxel_size : [0.1, 0.1, 4]
      max_number_of_points_per_voxel : 100
    }
    voxel_feature_extractor: {
      module_class_name: "PillarFeatureNet"
      num_filters: [64]
      with_distance: false
      num_input_features: 5
    }
    middle_feature_extractor: {
      module_class_name: "PointPillarsScatter"
      downsample_factor: 1
      num_input_features: 64
    }
    rpn: {
      module_class_name: "RPNV2"
      layer_nums: [3, 5, 5]
      layer_strides: [1, 2, 2]
      num_filters: [64, 128, 256]
      upsample_strides: [1, 2, 4]
      num_upsample_filters: [128, 128, 128]
      use_groupnorm: false
      num_groups: 32
      num_input_features: 64
    }
    loss: {
      classification_loss: {
        weighted_sigmoid_foca

<ipython-input-24-ad533a793f8e>:25: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for example in tqdm_notebook(dataloader):


  0%|          | 0/3125 [00:00<?, ?it/s]

tensor([0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0')
tensor([0.8405, 0.8049, 0.7980, 0.7876, 0.7805, 0.7758, 0.7666, 0.7649],
       device='cuda:0')
step=150, epoch=0.048, steptime=1.195, valid=0.0, loss=2.511, cls_loss=0.2831, loc_loss=2.228, lr=0.0001014, eta= 0 hr 00 m
Model saved: /kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874/29-1-2021_17:30, 150
########################
 EVALUATE
########################
Generating eval images...
eval images saved at /kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874/29-1-2021_17:30/images/150/
eval images saved at /kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874/29-1-2021_17:30/images/150/
eval images saved at /kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874/29-1-2021_17:30/images/150/
eval images saved at /kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874/29-1-2021_17:30/images/150/
eval ima

eval images saved at /kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874/29-1-2021_17:30/images/750/
eval images saved at /kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874/29-1-2021_17:30/images/750/
eval images saved at /kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874/29-1-2021_17:30/images/750/
eval images saved at /kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611755918.3152874/29-1-2021_17:30/images/750/
tensor([0, 0, 1, 0, 1, 0], device='cuda:0')
tensor([0.8670, 0.8403, 0.7921, 0.7356, 0.6938, 0.6579], device='cuda:0')
generate eval images finished(268.46/s). continue training:
tensor([0, 0, 0, 1, 1, 0, 0, 0, 1], device='cuda:0')
tensor([0.8840, 0.8606, 0.7807, 0.7766, 0.7766, 0.7715, 0.7491, 0.7472, 0.7441],
       device='cuda:0')
step=800, epoch=0.256, steptime=1.363, valid=0.0, loss=2.416, cls_loss=0.2334, loc_loss=2.182, lr=0.0001388, eta= 0 hr 14 m
tensor([0, 0, 0, 0, 

KeyboardInterrupt: 

In [ ]:
with open('/media/starlet/LdTho/data/sets/nuscenes/v1.0-trainval/nusc_custom_dbinfos_train.pkl', 'rb') as f:
    cus_info_file = pickle.load(f)
with open('/media/starlet/LdTho/data/sets/nuscenes/v1.0-trainval/infos_train.pkl', 'rb') as f:
    info_train = pickle.load(f)

In [ ]:
loss

## Save model

## Visualizing

In [25]:
net.eval()

VoxelNet(
  (voxel_feature_extractor): PillarFeatureNet(
    (pfn_layers): ModuleList(
      (0): PFNLayer(
        (linear): DefaultArgLayer(in_features=10, out_features=64, bias=False)
        (norm): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
    )
  )
  (middle_feature_extractor): PointPillarsScatter()
  (rpn): RPNV2(
    (blocks): ModuleList(
      (0): Sequential(
        (0): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        (1): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (2): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

In [80]:
# example = next(iter(dataloader))
example = next(iter(eval_dataloader))

example = example_convert_to_torch(example, float_dtype)
detection = net(example)
print(detection)

filtered_sample_tokens = eval_dataset.dataset.filtered_sample_tokens
# filtered_sample_tokens = dataset.dataset.filtered_sample_tokens


index = filtered_sample_tokens.index(detection[0]['metadata']['token'])
det_boxes = detection[0]['box3d_lidar'].cpu().detach().numpy()
det_labels = detection[0]['label_preds'].cpu().detach().numpy()
det_scores = detection[0]['scores'].cpu().detach().numpy()

det_boxes_cones = det_boxes[det_labels == 0]
det_scores_cones = det_scores[det_labels == 0]
det_labels_cones = det_labels[det_labels == 0]

det_boxes_pedes = det_boxes[det_labels == 1]
det_scores_pedes = det_scores[det_labels == 1]
det_labels_pedes = det_labels[det_labels == 1]

# gt_example = dataset.dataset.get_sensor_data(index)
gt_example = eval_dataset.dataset.get_sensor_data(index)

points = gt_example['lidar']['points']

gt_boxes = gt_example['lidar']['annotations']['boxes']
gt_labels = gt_example['lidar']['annotations']['names']
gt_boxes_cones = gt_boxes[gt_labels == "traffic_cone"]
gt_labels_cones = gt_labels[gt_labels == "traffic_cone"]

gt_boxes_pedes = gt_boxes[gt_labels == "pedestrian"]
gt_labels_pedes = gt_labels[gt_labels == "pedestrian"]

gt_scores = np.ones(len(gt_labels_cones))
c = points[:, 3].reshape(-1, 1)
c = np.concatenate([c, c, c], axis=1)
points = points[:, 0:3]
pc = o3d.geometry.PointCloud()
pc.points = o3d.utility.Vector3dVector(points)
pc.colors = o3d.utility.Vector3dVector(c)
mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(size=2.0,
                                                               origin=[-0, -0, -0])
geo = [pc, mesh_frame]
rbbox_corners_cones = box_np_ops.center_to_corner_box3d(det_boxes_cones[:, :3],
                                                  det_boxes_cones[:, 3:6],
                                                  det_boxes_cones[:, 6],
                                                  origin=(0.5, 0.5, 0.5), axis=2)
rbbox_corners_pedes = box_np_ops.center_to_corner_box3d(det_boxes_pedes[:, :3],
                                                  det_boxes_pedes[:, 3:6],
                                                  det_boxes_pedes[:, 6],
                                                  origin=(0.5, 0.5, 0.5), axis=2)
gt_cones_rbbox_corners = box_np_ops.center_to_corner_box3d(gt_boxes_cones[:, :3],
                                                     gt_boxes_cones[:, 3:6],
                                                     gt_boxes_cones[:, 6],
                                                     origin=(0.5, 0.5, 0.5), axis=2)
gt_pedes_rbbox_corners = box_np_ops.center_to_corner_box3d(gt_boxes_pedes[:, :3],
                                                     gt_boxes_pedes[:, 3:6],
                                                     gt_boxes_pedes[:, 6],
                                                     origin=(0.5, 0.5, 0.5), axis=2)
for i in range(len(rbbox_corners_cones)):
    geo.append(buildBBox(rbbox_corners_cones[i], color=[1, 0, 0]))
for i in range(len(rbbox_corners_pedes)):
    geo.append(buildBBox(rbbox_corners_pedes[i], color=[1, 0, 1]))
for i in range(len(gt_cones_rbbox_corners)):
    geo.append(buildBBox(gt_cones_rbbox_corners[i], color=[0, 1, 0]))
for i in range(len(gt_pedes_rbbox_corners)):
    geo.append(buildBBox(gt_pedes_rbbox_corners[i], color=[0, 0, 1]))

[{'box3d_lidar': tensor([[ 6.7495, -4.7776,  0.2637,  0.3864,  0.3830,  0.7659, -1.5742],
        [ 9.5076, -7.3982,  0.1035,  0.3867,  0.3853,  0.7745, -1.5293],
        [12.0970, -4.8765,  0.2988,  0.3835,  0.3754,  0.7661, -1.5879],
        [ 8.3111, -7.4358,  0.1504,  0.3699,  0.3720,  0.7410,  1.4990],
        [ 5.7080, -7.5948,  0.2344,  0.3796,  0.3737,  0.7455, -1.5410],
        [14.4314, -7.0159,  0.2207,  0.3850,  0.3788,  0.7692, -1.5527],
        [11.1903, -7.3256,  0.2227,  0.3837,  0.3791,  0.7678, -1.5547],
        [12.7240, -7.2016,  0.2285,  0.3773,  0.3759,  0.7586, -1.5664],
        [17.6695, -6.6163,  0.2383,  0.3776,  0.3738,  0.8115, -1.5117]],
       device='cuda:0'), 'scores': tensor([0.8882, 0.8473, 0.8468, 0.8393, 0.8364, 0.8302, 0.8300, 0.8253, 0.8142],
       device='cuda:0'), 'label_preds': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0], device='cuda:0'), 'metadata': {'token': 'da08b54635664d7cbadadc3ca90a0db7'}}]


In [81]:
o3d.visualization.draw_geometries(geo)

In [23]:
net.train()

VoxelNet(
  (voxel_feature_extractor): PillarFeatureNet(
    (pfn_layers): ModuleList(
      (0): PFNLayer(
        (linear): DefaultArgLayer(in_features=10, out_features=64, bias=False)
        (norm): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
    )
  )
  (middle_feature_extractor): PointPillarsScatter()
  (rpn): RPNV2(
    (blocks): ModuleList(
      (0): Sequential(
        (0): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        (1): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(1, 1), bias=False)
        (2): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

In [ ]:
eval_dataset[1]